# Amazon 상품 Review 데이터 가져오기
* Selenium과 BeautifulSoup을 이용

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import os, warnings 
import re
warnings.filterwarnings(action='ignore') #warning message 무시하고 숨기기

### 특정 상품 url로 찾아가기

In [9]:
driver = webdriver.Chrome('chromedriver')
url='https://www.amazon.com/dp/B07QFB9SV2/ref=sspa_dk_detail_5?psc=1&pd_rd_i=B07QFB9SV2&pd_rd_w=LvcTb&pf_rd_p=5d846283-ed3e-4512-a744-a30f97c5d738&pd_rd_wg=VLjs8&pf_rd_r=MBVK8GCEJ6A68WKE26VC&pd_rd_r=b0b15273-a074-46aa-aaa7-3c0577ab1b76&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUExQ0NISzRWWk1aTVpPJmVuY3J5cHRlZElkPUEwNzI1OTM0MlJRSEFNMUtOSDJVWCZlbmNyeXB0ZWRBZElkPUEwNTYwNjQ5M0tBUUgwWThTOVhWVSZ3aWRnZXROYW1lPXNwX2RldGFpbF90aGVtYXRpYyZhY3Rpb249Y2xpY2tSZWRpcmVjdCZkb05vdExvZ0NsaWNrPXRydWU='
driver.get(url)

### 리뷰 클릭하고 전체 리뷰 보기

In [10]:
reviewSee = driver.find_element_by_xpath('//*[@id="acrCustomerReviewText"]')
reviewSee.click()

reviewAll = driver.find_element_by_xpath('//*[@id="cr-pagination-footer-0"]/a')
reviewAll.click()

### review 정보 가지고 오기
* review 내용 , 작성자, 평점, 작성날짜, 작성지역, 유용함 정도
* 10개 페이지 내용 가지고 오기

In [11]:
review_content=[]
reviewer =[]
review_rating=[]
review_date=[]
review_area=[]
review_helpful=[]

for ck in range(10):
    page = driver.page_source
    soup = BeautifulSoup(page,'lxml')
    
    all_review_contents = soup.select('span[data-hook=review-body]')
    all_reviewer = soup.select('div[id=cm_cr-review_list] span[class=a-profile-name]')
    all_rate = soup.select('i[data-hook=review-star-rating]')
    all_date = soup.select('span[data-hook=review-date]')
    all_review_helpful=soup.find_all("div", class_="a-section review aok-relative")
    
    for j in all_review_contents:
        review_content.append(j.text.strip())

    for k in all_reviewer:
        try:
            reviewer.append(k.text.strip())
        except:
            reviewer.append('0')
    for l in all_rate:
        try:
            review_rating.append(l.text.strip()[0:3])
        except:
            review_area.append('0')
        
        #공백으로 split해서 가지고 올 수도 있다.
        #정규 표현식을 사용 할 수도 있다.
        #print(re.findall("\d+",l.text)[0])#여기서 숫자인 부분만 뽑겠다.

    for m in all_date:
        try:
            review_date.append(" ".join(m.text.split()[6:]))
        except:
            review_area.append('0')
            
        try:    
            #on이라는 문자로 나눠주기
            #print(m.text.split("on")[-1].strip())
            review_area.append(" ".join(m.text.split()[3:5]))
        except:
            review_are.append('0')
            
    for o in all_review_helpful:
        try:
            help_count=o.find("span", class_="a-size-base a-color-tertiary cr-vote-text").text.split()[0]
            if help_count == 'One':
                review_helpful.append(1)
            else    :        
                review_helpful.append(help_count)
        except:
            review_helpful.append(0)

    nextPage=driver.find_element_by_xpath('//*[@id="cm_cr-pagination_bar"]/ul/li[2]/a')
    nextPage.click()    
    


### 데이터 프레임 만들어 csv파일 생성

In [14]:
data ={'review': review_content, 
       'reviewer':reviewer, 
       'rate':review_rating ,
       'review_date': review_date,
       'area':review_area,
       'helpful_count':review_helpful}

data = pd.DataFrame(data)
display(data)
data.to_csv('shoppingmall_review.csv',index=False, encoding='utf-8-sig')

,review,reviewer,rate,review_date,area,helpful_count
0,I got this for my nephew who just turned 2yo. ...,LeeAnne,5.0,"December 27, 2019",United States,12
1,Much smaller than expected. It’s very cheaply ...,lisa,3.0,"February 13, 2020",United States,7
2,Be very careful if kids pick this up the wrong...,The Curl Girl,3.0,"February 18, 2021",United States,2
3,My 3 year old likes it which is what matters t...,JESSIE,4.0,"February 17, 2020",United States,4
4,Little guy loves trucks and this is probably h...,Adam,5.0,"December 25, 2019",United States,5
5,My 3 year old grandson loves this truck. Lots ...,Sheree Hagadone,5.0,"May 5, 2020",United States,4
6,My grandson played for hours with an old CAT d...,Carol Clark,1.0,"January 3, 2020",United States,4
7,Bought for Christmas presents! The kids will l...,Amazongirl,5.0,"November 13, 2019",United States,5
8,My 3 year old nephew plays hard with his toys ...,Babzy,5.0,"October 22, 2020",United States,0
9,This is a great item. Very durable and my kid ...,Matt,5.0,"July 7, 2020",United States,1
